In [1]:
import sys
import argparse
import numpy as np
import time
import zipfile
import math
import textwrap
import pprint as pprint

from collections import defaultdict, Counter, OrderedDict

ref_raw = '/content/sample_1000_reference_genome.fasta'
reads_raw = '/content/sample_1000_with_error_paired_reads.fasta'

refs_file = '/content/sample_1000_reference_genome.fasta'
reads_file = '/content/sample_1000_with_error_paired_reads.fasta'

def reads_format(reads_file):
    try:
        with open(reads_file, 'r') as read:
            first_line = True
            count = 0
            reads = []
            for line in read:
                count += 1
                if first_line:
                    first_line = False
                    continue
                ends = line.strip().split(',')
                reads.append(ends)
        return reads
    except IOError:
        print("error reading ", reads_file)
        return None

def ref_format(ref_fn):
    try:
        with open(ref_fn, 'r') as genome:
            first_line = True
            ref_genome = ''
            for line in genome:
                if first_line:
                    first_line = False
                    continue
                ref_genome += line.strip()
        return ref_genome
    except IOError:
        print("error reading ", ref_fn)
        return None

# ref = formatted file
def refDict(ref, k):
    d = {}
    for i in range(len(ref)-k):
        seq = ref[i:i+k]
        d.setdefault(seq, []).append(i)
    return d
# ^ create genome data structure dict for k-mer

In [2]:
k_mer_maps = {}
nucleotides = ['A', 'C', 'G', 'T']
for i in range(53):
    k_mer_maps[i] = refDict(ref_format(refs_file), i)

error reading  /content/sample_1000_reference_genome.fasta


TypeError: ignored

In [ ]:
def align(read, n = 3): #breaks up the reads into n sections
  readsArray = []
  length = len(read)
  if n == 3:
    return read[:length // n], read[length // 3:2 * length // 3], read[2 * length // 3:length]
  else:
    for i in range(1, n): #goes up to n - 1
    # this is for length % n == n-1
      if length % 3 == i:
       return read[:length // n + (i - 1)], read[length // n+i-1 : 2*length // n + i], read[2*length // n + i:length]

# NEED TO WRITE GENERAL CODE:
# split string of arbitrary length into arbitrary # of sections
# return array of reads (right now it returns the strings themselves, so
# if we return an array I'll need to edit the code in substitutions() that depends on the 3 parts of the read)

In [ ]:
def hamming_distance(p, q):
  return sum([(1 if p[i] != q[i] else 0) for i in range(len(p))])
  
def neighbors(read, d = 2):
  if d == 0:
    return {read}
  if len(read) == 1:
    return {'A', 'C', 'G', 'T'}
  neighborhood = set()
  suffix_neighbors = neighbors(read[1:], d)
  for text in suffix_neighbors:
    if hamming_distance(read[1:], text) < d:
      for i in ['A', 'C', 'G', 'T']:
        neighborhood.add(i + text)
    else:
      neighborhood.add(read[0] + text)
  return neighborhood

neighbors('TATCTACACATTACGAG')

for i in range(len(read_f)):
  read = reads_notitle[i][0]
  readNeighbor = neighbors(read, 1) #compute the 1-neighborhood of a read


In [ ]:
# find keys of substitutions for a given read
def substitutions(reads_file, reference_genome_file, threshold = 4):

  reads = reads_format(reads_file)
  reference_genome = ref_format(reference_genome_file)
  differences = {}
  alignment_table = [k_mer_maps]

  # tried to make alignment_table into a dict for faster access
  # failed
  # alignment_dict = {}
  # for key in k_mer_maps.keys():
  #   alignment_dict[key] = (dict(zip(frozenset(k_mer_maps[key]))))
  
  genome_length = len(reference_genome)

  whichread = 0

  numSplits = 4 #testing splitting into arbitrary n sections
  read_sections2 = align(reads[0::2], numSplits) # should only be passing in contents, not titles. not sure how below code takes care of that. 
  print(len(read_sections2))
  # print("read_sections2[0] at 4 splits: ", read_sections2[0])
  # print("read_sections2[1] at 4 splits: ", read_sections2[1])
  # print("read_sections2[2] at 4 splits: ", read_sections2[2]) #should be third

  #iterate thru all reads
  for read_pair in reads: # READ PAIR? !!!!!!
    for read in read_pair:
      length = len(read)
      if read in k_mer_maps[length].keys(): #look up FULL read in genome data struct to see if it's perfect
        alignment_table.append('-' * k_mer_maps[length][read][0] + read + '-' * (genome_length - (k_mer_maps[length][read][0] + length)))
        continue

      # split up read into n sections
      read_sections = align(read) 
      # temp = []
      # for i in range(numSplits):
      #   temp.append(len(read_sections[i]))
      # read_section_lengths = tuple(temp)
      # print(read_section_lengths)
      read_section_lengths = (len(read_sections[0]), len(read_sections[1]), len(read_sections[2]))

      valid_sections = [[], [], []] # make array of valid sections
      for i in range(3): #numsplits = 3
        for neighbor_read in neighbors(read_sections[i], 2):
          if neighbor_read in k_mer_maps[read_section_lengths[i]].keys():
            for position in k_mer_maps[read_section_lengths[i]][neighbor_read]:
              valid_sections[i].append((neighbor_read, position))
              # if a substring of read is valid append it
      
      # if indices of each valid section create full length read
      # then add to array of complete sequence finds
      complete_sequence_finds = []
      for find_a in valid_sections[0]:
        for find_b in valid_sections[1]:
          if find_a[1] + read_section_lengths[0] == find_b[1]:
            for find_c in valid_sections[2]:
              if find_b[1] + read_section_lengths[1] == find_c[1]:
                complete_sequence_finds.append((find_a[0] + find_b[0] + find_c[0], find_a[1]))
      
      # for each entry in complete sequence finds, ask if it's a substitution
      for find in complete_sequence_finds:
        referece_genome_portion = find[0] #string
        portion_location = find[1] #position
        
        alignment_table.append('-' * portion_location + read + '-' * (genome_length - (portion_location + length)))
        substitutions = [(portion_location + i, referece_genome_portion[i], read[i]) for i in range(length) if read[i] != referece_genome_portion[i]]
        for sub in substitutions:
          differences[sub] = differences.get(sub, 0) + 1 #append to differences dict. if it already exists, iterate, if not, initialize with 0 (i think)

  return sorted([(key, differences[key]) for key in differences.keys() if differences[key] > threshold])
  # if differences[key] above threshold we're convinced it's not an error

In [ ]:
output = substitutions(reads_raw, ref_raw)

def pprint_1a(output):
  for (ind, orig, new), val in output:
    print(">S" + str(ind), orig, new)

pprint_1a(output)